#6-2. Lightning 예제 (MNIST)


앞의 신경망 모델 설명에서 Fashion MNIST 를 신경망 모델로 풀어 보았다. 이제 동일한 작업을 PyTorch-Lightning 으로 풀어보겠습니다.

https://wikidocs.net/edit/page/156984 https://wikidocs.net/edit/page/156984

FashionMNIST 데이터를 가져와서 DataLoader 를 만드는 코드는 앞장의 내용과 동일 합니다. 다른 점은 train 데이터셋의 일부를 random으로 샘플링해 validation 용도로 사용하겠습니다.

ligthning module을 정의하고 training, validation, test 그리고 optimizer 등을 구현해야 합니다.

init 에서는 신경망 모델을 구성해 줍니다. 앞장의 예제와 동일하게 Linear, BatchNorm1d, ReLU 순서로 Layer를 쌓아줍니다. 각각의 Layer에 대한 설명은 앞장을 참고 하세요.



In [3]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.8 MB/s eta 0:00:00


In [18]:
!pip install torchmetrics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core import LightningModule
from torchmetrics import functional as FM
import matplotlib.pyplot as plt

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn.functional as F

def __init__(self):
  super().__init__()
  self.model = nn.Sequential(
      nn.Flatten(),
      nn.Linear(28*28, 512),
      torch.nn.BatchNorm1d(512),
      nn.ReLU(inplace=True),
      nn.Linear(512, 256),
      torch.nn.BatchNorm1d(256),
      nn.ReLU(inplace=True),
      nn.Linear(256, 64),
      torch.nn.BatchNorm1d(64),
      nn.ReLU(inplace=True),
      nn.Linear(64, 10)
  )

forward는 모델의 예측 결과를 제공하고 싶을 때 사용합니다. argument로 받은 x를 init에서 정의한 모델로 전달하여 그 결과를 리턴합니다.



In [6]:
def forward(self, x):
  return self.model(x)

training_step은 학습 루프의 body 부분을 나타냅니다. 이 메소드에서는 argument로 training 데이터로더가 제공하는 batch와 해당 batch의 인덱스가 전달 됩니다. training_step에서는 손실함수를 사용하여 loss를 계산하고 리턴하면 됩니다. 모델의 output과 정답 라벨 사이의 cross entropy loss를 구해서 넘겨줍니다.



In [7]:
def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss


validation_step은 학습 중간에 모델의 성능을 체크하는 용도로 사용합니다. argument로 training 데이터로더가 제공하는 batch와 해당 batch의 인덱스가 전달 됩니다. accuracy 함수는 pytorch_lightning.metrics.functional에서 정의되어 있는 함수로 예측한 y_pred값이 실제 y값과 일치하는 비율을 구해줍니다. 하나의 값을 표시할 때는 self.log(<변수 이름="">, <값>)과 같이 할 수 있고 여러 개의 변수를 표시하고 싶으면 아래와 같이 self.log_dict로 변수 이름, 값 쌍을 가지고 있는 딕셔너리를 표시할 수 있습니다.



In [8]:
def validation_step(self, batch, batch_idx):
        x, y = batch
        pred_y = self(x)
        acc = FM.accuracy(pred_y, y)
        loss = F.cross_entropy(pred_y, y)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)


test_step은 앞의 test DataLoader에서 제공하는 batch 데이터를 가지고 확인하고 싶은 통계량을 기록하는데 사용할 수 있습니다. 제공되는 batch 데이터만 다를 뿐 코드는 validation_step과 동일 합니다.



In [9]:
def test_step(self, batch, batch_idx):
        x, y = batch
        pred_y = self(x)
        acc = FM.accuracy(pred_y, y)
        loss = F.cross_entropy(pred_y, y)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)

configure_optimizers에서는 모델의 optimizer와 scheduler를 구현합니다. 앞장과 동일하도록 Adam optimzer를 사용하도록 하겠습니다.



In [10]:
def configure_optimizers(self):
        optimizer = Adam(self.parameters())
        return optimizer

이제 학습을 하고 테스트를 하면 됩니다. ytorch lightning에서 학습을 위해 추가로 작성해야 할 코드는 매우 짧습니다.

모델을 학습하기 위해서는 학습 로직을 정하는 Trainer를 생성해야 합니다. gpus가 0일 때는 cpu를 사용하고 gpus가 1 이상이면 gpu를 사용하여 모델을 학습합니다. gpus가 2 이상이면 자동으로 다중 gpu를 활용해 분산 학습을 진행하게 됩니다.

Trainer와 lightning module을 정의하고 난 뒤에 Trainer의 fit 함수로 모델을 학습할 수 있습니다. fit의 파라미터로 모델, training 데이터로더와 validation 데이터로더를 넘겨줍니다. test 함수에 test 데이터로더를 넘겨주어 모델을 테스트할 수 있습니다.



다음과 같은 학습 결과를 얻을 수 있습니다.



In [11]:
model = LitModel()
trainer = Trainer(max_epochs=epochs, gpus=1)
trainer.fit(model, train_dataloader, val_dataloader)
trainer.test(test_dataloaders=test_dataloader)


NameError: ignored

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from pytorch_lightning import LightningModule, Trainer
import pytorch_lightning.metrics.functional as FM
import matplotlib.pyplot as plt

# 데이터셋 다운로드 및 전처리
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분리
train_dataset, val_dataset = random_split(training_data, [55000, 5000])

# 하이퍼파라미터 설정
learning_rate = 1e-3
batch_size = 64
epochs = 10

# 데이터 로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# 모델 정의
class LitModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 512),
            torch.nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 64),
            torch.nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10)
        )

    # 순전파 함수
    def forward(self, x):
        return self.model(x)

    # 훈련 스텝
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    # 검증 스텝
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = FM.accuracy(logits, y)
        loss = F.cross_entropy(logits, y)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)

    # 테스트 스텝
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = FM.accuracy(logits, y)
        loss = F.cross_entropy(logits, y)
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics)    

    # 옵티마이저 설정
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

# 모델 생성 및 훈련
model = LitModel()
trainer = Trainer(max_epochs=epochs, gpus=0)
trainer.fit(model, train_dataloader, val_dataloader)

# 모델 테스트
trainer.test(test_dataloaders=test_dataloader)

# 레이블 및 클래스 목록
label_tags = {
    0: 'T-Shirt', 
    1: 'Trouser', 
    2: 'Pullover', 
    3: 'Dress', 
    4: 'Coat', 
    5: 'Sandal', 
    6: 'Shirt',
    7: 'Sneaker', 
    8: 'Bag', 
    9: 'Ankle Boot'
}

# 예측 결과를 시각화하기 위한 설정
columns = 6
rows = 6
fig = plt.figure(figsize=(10,10))

# 모델 평가 모드
model.eval()

# 예측 결과를 그리드 형태로 시각화
for i in range(1, columns*rows+1):
    data_idx = torch.randint(len(test_dataloader), (1,)).item()
    input_img = test_data[data_idx][0].unsqueeze(dim=0)

    output = model(input_img)
    _, argmax = torch.max(output, 1)
    pred = label_tags[argmax.item()]
    label = label_tags[test_data[data_idx][1]]

    fig.add_subplot(rows, columns, i)
    if pred == label:
        plt.title(pred)
        cmap = 'Blues'
    else:
        plt.title(pred + '=>' +  label)
        cmap = 'Reds'
    plot_img = test_data[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')

# 그림 표시
plt.show() 


TypeError: ignored